In [1]:
import cv2
import mediapipe as mp
import pyautogui as pg

In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)
mpDraw = mp.solutions.drawing_utils

In [3]:
cr_x, cr_y = 0, 0
pr_x, pr_y = 0, 0

In [4]:
def cal_distance(Pts):
    distance = pow(Pts[2]-Pts[0], 2) + pow(Pts[3]-Pts[1], 2)
    return distance

In [5]:
camera = cv2.VideoCapture(0)
camera.set(3, 1280)
camera.set(4, 720)
while True :
    check, image = camera.read()
    img_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(img_RGB)
    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            Pts = []
            for id, lms in enumerate(hand.landmark):
                h, w, c = image.shape
                cx, cy = int(lms.x*w), int(lms.y*h)
                if (id==12):
                    Pts.append(cx)
                    Pts.append(cy)
                    dist = cal_distance(Pts)
                    if (dist < 1200):
                        pg.click(1366 - cr_x, cr_y)
                if (id==8):
                    Pts.append(cx)
                    Pts.append(cy)
                    cr_x = pr_x + (cx - pr_x) / 2
                    cr_y = pr_y +  (cy - pr_y) / 2
                    if (cr_x < 683):
                        cr_x = cr_x - 0.292*(683 - cr_x)
                        if(cr_x < 0):
                            cr_x = 4
                    elif (cr_x > 683):
                        cr_x = cr_x + 0.292*(cr_x - 683)
                        if(cr_x > 1366):
                            cr_x = 1355
                    if (cr_y < 384):
                        cr_y = cr_y - 0.52*(384 - cr_y)
                        if(cr_y < 0):
                            cr_y = 4
                    elif (cr_y > 384):
                        cr_y = cr_y + 0.52*(cr_y - 384)
                        if(cr_y > 768):
                            cr_y =755           
                    pg.moveTo(1366 - cr_x, cr_y, duration=0.1)         
            mpDraw.draw_landmarks(image, hand, mpHands.HAND_CONNECTIONS)
            pr_x, pr_y = cr_x, cr_y
    cv2.imshow("Hands", image)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
camera.release()
cv2.destroyAllWindows()